## Comparing best agent's performance with classical baseline

### Three Steps

1. Identify the checkpoint with best evaluation performance.
2. Restore the agent with identified checkpoint.
3. Run the restored agent on 100000 episodes with the same seed used for computing baseline.

### Step 2: Restoring agent from checkpoint

In [1]:
from pathlib import Path

from ray.rllib.agents.ppo import PPOTrainer
from ray.tune import ExperimentAnalysis
from ray.tune.registry import register_env

from inventory_env.env_creator import inventory_env_creator

register_env("inventory_env", inventory_env_creator)

path_to_results_dir = Path().absolute().parent / "inventory_management_rl" / "experiments" / "experiment_results" / "experiment_wrappers"
analysis = ExperimentAnalysis(path_to_results_dir, default_metric="evaluation/episode_reward_mean", default_mode="max")
best_trial = analysis.get_best_trial(scope="all")
best_checkpoint = analysis.get_best_checkpoint(best_trial)
config = best_trial.config
agent = PPOTrainer(config=config)
agent.restore(best_checkpoint)

2023-04-27 17:09:46,639	WARNING deprecation.py:45 -- DeprecationWarning: `evaluation_num_episodes` has been deprecated. Use ``evaluation_duration` and `evaluation_duration_unit=episodes`` instead. This will raise an error in the future!
(RolloutWorker pid=1732) /home/dibya/miniconda3/envs/real_world_deep_rl_course_cpu/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=1732)   logger.warn(
2023-04-27 17:09:52,515	WARNING deprecation.py:45 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
/home/dibya/miniconda3/envs/real_world_deep_rl_course_cpu/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
2023-04-27 17:09:53,174	WARNING util.py:55 -- Install gputil for GPU system monitoring.
2023-04-27 17:09:53,237	INFO trainable.py:495 -- Restored on 192.168.0.178 from checkpoi

### Step 3: Run restored agent for 100000 episodes with seed 0

In [2]:
import numpy as np

env = inventory_env_creator(
    {
        "obs_filter": "my_normalize", 
        "reward_filter": None, 
    }, 
    seed=0
)
num_episodes = 100000
all_r = []
for _ in range(num_episodes):
    obs = env.reset()
    ep_r = 0
    while True:
        action = agent.compute_action(obs, unsquash_action=True)
        obs, r, done, _ = env.step(np.around(action))
        ep_r += r
        if done:
            break
    all_r.append(ep_r)
baseline = sum(all_r) / num_episodes
print(baseline)

2023-04-07 19:39:19,332	WARNING deprecation.py:45 -- DeprecationWarning: `compute_action` has been deprecated. Use `Trainer.compute_single_action()` instead. This will raise an error in the future!


175694.70564031467
